# Exercices Pratiques Processus

## Processus et tourniquet

In [ ]:
def tourniquet(file, quantum):
    ordre = []
    while not file.est_vide():
        processus = file.defile()
        ordre.append(processus.pid)
        processus.execute(quantum)
        if not processus.est_fini():
            file.enfile(processus)
    return ordre

## Etat d'un processus

In [ ]:
class Processus():

    def __init__(self, nom, duree, arrivee=0):
        self.etat = "non créé"
        self.nom = nom
        self.duree = duree
        self.arrivee = arrivee
        self.temps_aloue = 0

    def elit(self, quantum):
        self.temps_alloue = quantum
        self.etat = "élu"

    def execute(self):
        self.duree = self.duree - 1
        self.temps_alloue = self.temps_alloue - 1
        if self.duree == 0:
            self.etat = "terminé"
        elif self.temps_alloue == 0:
            self.etat = "prêt"

    def est_pret(self, horloge):
        if self.arrivee == horloge:
            self.etat = "prêt"
        return self.etat == "prêt"

    def bloque(self):
        self.etat = "bloqué"

    def debloque(self):
        self.etat = "prêt"


    def __repr__(self):
        return str({ "etat": self.etat, "nom":self.nom, "arrivee" :self.arrivee, "duree":self.duree})

## Ordonnancement de processus

In [ ]:
QUANTUM = 2
def temps_execution(arrivees, quantum):
    tps_execution = {}
    temps_horloge = 0
    file_attente = File()
    terminee = False
    processus_courant = None
    while not (arrivees.est_vide() and file_attente.est_vide() and processus_courant == None):

        if  not arrivees.est_vide() and arrivees.tete().est_pret(temps_horloge):
            file_attente.enfile(arrivees.defile())

        if processus_courant != None and processus_courant.etat == "prêt":
                file_attente.enfile(processus_courant)
                processus_courant = None

        if processus_courant == None and not file_attente.est_vide():
            processus_courant = file_attente.defile()
            processus_courant.elit(quantum)
            debut_quantun = temps_horloge 

        temps_horloge = temps_horloge + 1
        if processus_courant != None:
            processus_courant.execute()
            if processus_courant.etat != "élu":
                if processus_courant.nom in tps_execution:
                    tps_execution[processus_courant.nom].append((debut_quantun, temps_horloge))
                else:
                    tps_execution[processus_courant.nom] = [(debut_quantun, temps_horloge)]
                if processus_courant.etat == "terminé":
                    processus_courant = None


    return tps_execution